# Region of interest

In [ ]:
def crop_between_keypoints(img, x, y):
    height, width = img.shape
    
    x, y = int(x), int(y)
    
    # Calculate bounding box around the keypoint
    left = max(0, x - width // 2)
    right = min(width, x + width // 2)
    top = max(0, y - height // 2)
    bottom = min(height, y + height // 2)

    # Crop the image
    return img[top:bottom, left:right]


def crop_around_keypoint(img, keypoint, width, height):
    h, w = img.shape
    
    x, y = int(keypoint[0]), int(keypoint[1])
    
    # Calculate bounding box around the keypoint
    left = max(0, x - width // 2)
    right = min(w, x + width // 2)
    top = max(0, y - height // 2)
    bottom = min(h, y + height // 2)
    
    # Crop the image
    return img[top:bottom, left:right]

def graph_plot(study_id, series_id):
    train_label_combinations = pl.DataFrame()
    for row in train_label.iter_rows():
        if row[0]==study_id:
            print(pl.DataFrame(row[:3]).transpose())
            data = pl.DataFrame(row[:3]).transpose()
            train_label_combinations=pl.concat([train_label_combinations, data])
    print(train_label_combinations)
    
    #rename columns
    train_label_combinations = train_label_combinations.rename({"column_0":"study_id", "column_1":"series_id", "column_2":"instance_number"})
    #extract unique combination
    train_label_combinations = train_label_combinations.unique(subset=["study_id", "series_id", "instance_number"]).sort(["study_id", "series_id", "instance_number"])
    
    instance_number_list = train_label_combinations.filter((pl.col("study_id")==study_id) & (pl.col("series_id")==series_id)).get_column("instance_number")
    #instance_number_list

    for instance_number in instance_number_list:
        #print(instance_number)
        print(f"=====study_id:{study_id}, series_id:{series_id}, instance_number:{instance_number}=====")
        #read image
        ds = pydicom.read_file(f'{INPUT_DIR}/train_images/{study_id}/{series_id}/{instance_number}.dcm')
        #draw original image
        df_plt = train_label.filter(
            (pl.col('study_id')==study_id)
            &(pl.col('series_id')==series_id)
            &(pl.col('instance_number')==instance_number)
        )
        plt.subplot(1,2,1)
        plt.imshow(ds.pixel_array, cmap='bone')
        #plt.title(f"study_id:{study_id}, series_id:{series_id}, instance_number:{instance_number}")

        #draw original image + label
        #draw image
        df_plt = train_label.filter(
            (pl.col('study_id')==study_id)
            &(pl.col('series_id')==series_id)
            &(pl.col('instance_number')==instance_number)
        )
        plt.subplot(1,2,2)
        plt.imshow(ds.pixel_array, cmap='bone')
        #plt.title(f"study_id:{study_id}, series_id:{series_id}, instance_number:{instance_number}")
        #draw rabel
        for row in df_plt.iter_rows():
            plt.scatter(row[-2], row[-1], color='red')
        plt.show()

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
import pydicom as dicom

study_id, series_id = 4290709089, 3274612423
instance_number = 5
directory =f'{INPUT_DIR}/train_images/{study_id}/{series_id}/{instance_number}.dcm'

def paddingOrCroppingImage(directory):
    ds=dicom.dcmread(directory)
    
    dcm_data=ds.pixel_array
    # Example target size (height, width)
    target_height, target_width = 512, 512

    # Current image shape
    current_height, current_width = dcm_data.shape
    print(current_height, current_width)

    if current_height > target_height or current_width > target_width:
        crop_top = (current_height - target_height) // 2
        crop_bottom = crop_top + target_height
        crop_left = (current_width - target_width) // 2
        crop_right = crop_left + target_width

        cropped_image = dcm_data[crop_top:crop_bottom, crop_left:crop_right]
        return cropped_image
    else:
        # Calculate padding sizes
        pad_height = max(0, target_height - current_height)
        pad_width = max(0, target_width - current_width)

        # Calculate the padding to apply to each side
        pad_top = pad_height // 2
        pad_bottom = pad_height - pad_top
        pad_left = pad_width // 2
        pad_right = pad_width - pad_left


        # Apply padding with replicated borders
        padded_image = cv2.copyMakeBorder(
            dcm_data,
            pad_top,
            pad_bottom,
            pad_left,
            pad_right,
            cv2.BORDER_REPLICATE
        )

        # Update Dicom metadata
        ds.Rows, ds.Columns = padded_image.shape
        ds.PixelData = padded_image.tobytes()
        print(pad_top, pad_bottom, pad_left, pad_right)
        return padded_image
    

def reversed_images_cropping(image):
    current_height, current_width = image.shape
    target_height, target_width = 320, 320

    crop_top = (current_height - target_height) // 2
    crop_bottom = crop_top + target_height
    crop_left = (current_width - target_width) // 2
    crop_right = crop_left + target_width

    cropped_image = image[crop_top:crop_bottom, crop_left:crop_right]
    return cropped_image
    

dfs = train_label.filter(
            (pl.col('study_id')==study_id)
            &(pl.col('series_id')==series_id)
            &(pl.col('instance_number')==instance_number)
        )
for row in dfs.iter_rows():
        plt.scatter(row[-2], row[-1], color='red')

        
updated_image = paddingOrCroppingImage(directory)
reversed_image = reversed_images_cropping(updated_image)

plt.figure(figsize=(10, 5))  # Adjust figure size if needed

# Plot the updated (cropped/padded) image on the left
plt.subplot(1, 2, 1)
plt.imshow(updated_image, cmap='gray')  # Use 'gray' colormap for grayscale images
plt.title('Updated Image')
plt.axis('off')  # Turn off axis labels

# Plot the reversed image on the right
plt.subplot(1, 2, 2)
plt.imshow(reversed_image, cmap='gray')
plt.title('Reversed Image')
plt.axis('off')  # Turn off axis labels

# Show the plot with both images
plt.show()